In [10]:
import json
from typing import NamedTuple


from kfp import dsl
from kfp.v2 import compiler
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    InputPath,
    OutputPath,
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    ClassificationMetrics,
    Metrics,
)

from kfp.v2.google.client import AIPlatformClient
from view_demo.utils import get_project_id
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [2]:
project_id = 'pytorch-tpu-nfs'
location = 'us-central1'
dataset_id = 'view_dataset'
table_id = 'weather_time_series'

In [3]:
feature_names = [
    'Date_Time', 
    'p__mbar', 
    'Tpot__K',
    'Tdew__degC',
    'rh__percent',
    'VPmax__mbar',
    'VPact__mbar',
    'VPdef__mbar',
    'sh__g_per_kg',
    'H2OC__mmol_per_mol',
    'rho__gm_per_cubic_m',
    'wv__m_per_s',
    'max_w__vm_per_s',
    'wd__deg'
]
target_names = 'T__degC'

In [ ]:
@component(
    base_image=CONTAINER_URI,
    output_component_file=f'{SRC_ROOT}/preprocess/preprocess.yaml',
)
def view_preprocess(
    project_id: str,
    raw_dataset: str,
    out_dataset: Output[Dataset],
):
    from view_demo.preprocess import create_dataset
    return create_dataset(project_id=project_id, csv_path=raw_dataset)
    

In [4]:
@component(
    base_image=CONTAINER_URI,
    output_component_file=f'{SRC_ROOT}/train/train.yaml',
)
def view_train(
    project_id: str,
    input_dataset: Input[Dataset],
    out_model: Output,
):
    from google.cloud import aiplatform
    aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)
    job = aiplatform.CustomTrainingJob(
        display_name="train-abalone-dist-1-replica",
        script_path="training_script.py",
        container_uri="gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest",
        requirements=["gcsfs==0.7.1"],
        model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
    )
    model = job.run(ds, replica_count=1, model_display_name="abalone-model")
    return model
    

In [ ]:
@dsl.pipeline(
    name="view-test-pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def view_pipeline(
    project_id: str = PROJECT_ID,
    raw_dataset: str = 'gs://bench-datasets/jena_climate_2009_2016.csv',
):
    preprocess_task = view_preprocess(project_id=project_id, raw_dataset=raw_dataset)


In [ ]:
from kfp.v2 import compiler as v2compiler
v2compiler.Compiler().compile(pipeline_func=view_pipeline,
                              package_path='view_pipeline_spec.json')

In [ ]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID, 
    region=REGION, 
    )

In [ ]:
result = api_client.create_run_from_job_spec(
    job_spec_path="view_pipeline_spec.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    parameter_values={
    },
)